In [1]:
import numpy as np
import matplotlib.pyplot as plt
import autokeras as ak
import tensorflow as tf
import os 

In [2]:
data = np.load('/home/ML4NO/Data/n1000000_0910_all_flat.npz')

In [4]:
data_selection = 1 # 0 for all, 1 for lowE(<5GeV), 2 for high(>5GeV)

if data_selection == 0:
    data_all = np.column_stack([data['ve_dune'], data['vu_dune'], data['vebar_dune'], data['vubar_dune']])
elif data_selection == 1:
    data_all = np.column_stack([data['ve_dune'][:,:36], data['vu_dune'][:,:36], data['vebar_dune'][:,:36], data['vubar_dune'][:,:36]])
elif data_selection == 2:
    data_all = np.column_stack([data['ve_dune'][:,36:], data['vu_dune'][:,36:], data['vebar_dune'][:,36:], data['vubar_dune'][:,36:]])

target = np.column_stack([data['theta23'], data['delta']])

x_train = data_all[:10000]
y_train = target[:10000]

x_train2 = data_all[10000:900000]
y_train2 = target[10000:900000]

x_test = data_all[900000:]
y_test = target[900000:]

In [5]:
clf = ak.StructuredDataRegressor(overwrite=True, max_trials=50)
clf.fit(x_train, y_train,
           validation_split = 0.1,
           batch_size=64,
           epochs=20,
           verbose=1,
           shuffle = True
       )

Trial 6 Complete [00h 00m 53s]
val_loss: 144.77825927734375

Best val_loss So Far: 144.77825927734375
Total elapsed time: 00h 05m 21s

Search: Running Trial #7

Hyperparameter    |Value             |Best Value So Far 
structured_data...|False             |True              
structured_data...|False             |False             
structured_data...|2                 |2                 
structured_data...|1024              |1024              
structured_data...|0                 |0                 
structured_data...|256               |256               
regression_head...|0                 |0                 
optimizer         |adam_weight_decay |adam_weight_decay 
learning_rate     |0.1               |0.1               

Epoch 1/20
142/142 [==============================] - 5s 23ms/step - loss: 18504.2939 - mean_squared_error: 18504.2939 - val_loss: 4574.6328 - val_mean_squared_error: 4574.6328
Epoch 2/20
142/142 [==============================] - 2s 16ms/step - loss: 5465.7491 - mean

KeyboardInterrupt: 

In [ ]:
model = clf.export_model()
model.summary()

In [ ]:
model.fit(x_train2, y_train2,
           validation_split = 0.1,
           batch_size=64,
           epochs=20,
           verbose=1,
           shuffle = True
         )

In [ ]:
index = 1
while os.path.isfile('/home/ML4NO/ML_DUNE/Regression/models/0910_2params_{}.h5'.format(index)):
    index += 1
model.save('/home/ML4NO/ML_DUNE/Regression/models/0910_2params_{}.h5'.format(index))

In [ ]:
scale_steps = np.logspace(-3, 0, 30)
before_train_loss = []
after_train_loss = []

for scale in scale_steps:
    x_train2_gen = np.random.normal(x_train2, np.sqrt(x_train2)*scale)
    x_test_gen = np.random.normal(x_test, np.sqrt(x_test)*scale)

    before_train_loss.append(model.evaluate(x_test_gen, y_test)[0])

    model.fit(x_train2_gen, y_train2,
               validation_split = 0.1,
               batch_size=64,
               epochs=5,
               verbose=1,
               shuffle = True
             )

    after_train_loss.append(model.evaluate(x_test_gen, y_test)[0])

In [ ]:
model_index = index
index = 1
path = '/home/ML4NO/ML_DUNE/Regression/models_furthurTrain/0910_2params_{}_{}.h5'
while os.path.isfile(path.format(model_index, index)):
    index += 1
model.save(path.format(model_index, index))
outfile = {'scale_steps': scale_steps,
           'before_train_loss': before_train_loss,
           'after_train_loss' :after_train_loss}
np.save(file = '/home/ML4NO/ML_DUNE/Regression/models_furthurTrain/0910_2params_{}_{}_result.npy'.format(model_index, index),
        arr = outfile)

In [ ]:
x_test2_gen = np.random.poisson(x_test)

for i in range(10):
    x_train2_gen = np.random.poisson(x_train2)
    
    model.fit(x_train2_gen, y_train2,
              validation_split=0.1,
               batch_size=64,
               epochs=1,
               verbose=1,
               shuffle = True
             )
model.evaluate(x_test2_gen, y_test)

In [ ]:
furthur_index = index
index = 1
path = '/home/ML4NO/ML_DUNE/Regression/models_PoissonTrain/0910_2params_{}_{}_{}.h5'
while os.path.isfile(path.format(model_index, furthur_index, index)):
    index += 1
model.save(path.format(model_index, furthur_index, index))